In [8]:
import joblib
import pickle
from collections import Counter
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.utils.class_weight import compute_class_weight
from xgboost import XGBClassifier

from success_prediction.config import RAW_DATA_DIR, PROCESSED_DATA_DIR, MODELS_DIR

In [9]:
df = pd.read_csv(RAW_DATA_DIR / 'company_sample' / 'until_2020' / '2020_sample_encoded_features.csv')

/var/folders/ly/6qys5jn501gbqmkkkgmvpf0h0000gn/T/ipykernel_59478/3524526548.py:1: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(RAW_DATA_DIR / 'company_sample' / 'until_2020' / '2020_sample_encoded_features.csv')


In [13]:
df['locations']

0                   []
1                   []
2                   []
3                   []
4                   []
              ...     
129561    ['Svizzera']
129562              []
129563    ['Svizzera']
129564    ['Svizzera']
129565              []
Name: locations, Length: 129566, dtype: object

In [ ]:
cols_to_drop = [
    'uid', 'current_name', 'founding_name', 'current_legal_form', 'current_street', 'current_town', 'current_zip_code',
    'current_country', 'founding_street', 'founding_town', 'founding_zip_code', 'shab_id', 'founding_message',
    'combined_address', 'geocoded_address', 'company_url', 'founding_name_norm', 'people', 'locations', 'current_purpose', 'founding_purpose',
    'exit_date', 'merger_date', 'investment_date', 'subsidy_date', 'n_female_founders'
]
df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
col_order = [

    'ehraid',  # Company ID only for identification
    
    # Targets
    'target_inv_exit',
    'target_acquisition',
    'target_non_gov_investment',
    'target_inno_subsidy'

    # Basic firm features
    'founding_legal_form',  # Cat

    'legal_form_binary_3.0',  # Binary
    'legal_form_binary_4.0',
    'legal_form_binary_5.0',
    'legal_form_binary_6.0',
    'legal_form_binary_8.0',
    'legal_form_binary_10.0',
    'legal_form_binary_12.0',
    'legal_form_binary_13.0',
    'legal_form_binary_15.0',
    'legal_form_binary_16.0',
    'legal_form_binary_17.0',
    'legal_form_binary_19.0',

    'section_1_label',  # Top 3 NOGA Levels
    'division_1_label',
    'group_1_label',
    'class_1_label',

    'section_2_label',
    'division_2_label',
    'group_2_label',
    'class_2_label',

    'section_3_label',
    'division_3_label',
    'group_3_label',
    'class_3_label',

    'capital_chf',  # registered capital

    # Firm name features
    'firm_name_length',  
    'firm_name_swiss_ref',
    'firm_name_geog_ref',
    'firm_name_founder_match',
    'firm_name_male_match',
    'firm_name_female_match',

    # Address features
    'latitude',
    'longitude',
    'founding_bfs_code',
    'district_id',
    'canton_id', 
    'urban_rural',  # Cat
    'typology_9c',  # Cat
    'typology_25c',  # Cat

    'urban_rural_2.0',  # Binary
    'urban_rural_3.0',

    'typology_9c_12.0',  # Binary
    'typology_9c_13.0',
    'typology_9c_21.0',
    'typology_9c_22.0',
    'typology_9c_23.0',
    'typology_9c_31.0',
    'typology_9c_32.0',
    'typology_9c_33.0',

    'typology_25c_112.0',  # Binary
    'typology_25c_113.0',
    'typology_25c_121.0',
    'typology_25c_122.0',
    'typology_25c_123.0',
    'typology_25c_134.0',
    'typology_25c_136.0',
    'typology_25c_137.0',
    'typology_25c_216.0',
    'typology_25c_217.0',
    'typology_25c_226.0',
    'typology_25c_227.0',
    'typology_25c_235.0',
    'typology_25c_236.0',
    'typology_25c_237.0',
    'typology_25c_314.0',
    'typology_25c_316.0',
    'typology_25c_317.0',
    'typology_25c_325.0',
    'typology_25c_326.0',
    'typology_25c_327.0',
    'typology_25c_334.0',
    'typology_25c_335.0',
    'typology_25c_338.0',

    'population',

    'n_firms_same_address',
    'n_firms_within_1km',
    'n_firms_within_2.5km',
    'n_firms_within_10km',

    # Founder features
    'n_founders',
    'n_inscribed_firms',
    'pct_female_founders',
    'n_distinct_nationalities',
    'n_swiss_founders',
    'n_foreign_founders',
    'n_dr_titles',
    'n_founders_same_residence',
    'founder_fids',
    'n_founders_with_experience',
    'pct_founders_with_experience',
    'prior_failed',
    'prior_existing',

    # BPS features
    'bps_language',
    'bps_normalized',
    'bps_length',
    'bps_mean_word_length',
    'bps_length_quantiles_5',
    'bps_lix',
    'bps_min_word_freq_norm',
    'bps_max_word_freq_norm',
    'bps_freq_ratio_norm',
    'has_location',
    'has_male_name',
    'has_female_name',

    # Website features
    'n_pages',
    'total_text_len',
    'mean_text_len',
    'n_internal_links_mean',
    'n_external_links_mean',
    'n_languages',
    'dominant_language',

    # Control variables
    'days_of_prior_observations'
    'founding_date',
    'prediction_1_score',
    'prediction_2_score',
    'prediction_3_score',
]

In [15]:
list(df.columns)

['ehraid',
 'founding_legal_form',
 'current_purpose',
 'founding_purpose',
 'founding_bfs_code',
 'founding_date',
 'latitude',
 'longitude',
 'population',
 'canton_id',
 'district_id',
 'urban_rural',
 'typology_9c',
 'typology_25c',
 'capital_chf',
 'n_pages',
 'total_text_len',
 'mean_text_len',
 'n_internal_links_mean',
 'n_external_links_mean',
 'n_languages',
 'dominant_language',
 'class_1_label',
 'section_1_label',
 'prediction_1_score',
 'class_2_label',
 'section_2_label',
 'prediction_2_score',
 'class_3_label',
 'section_3_label',
 'prediction_3_score',
 'exit_date',
 'target_inv_exit',
 'merger_date',
 'target_acquisition',
 'investment_date',
 'target_non_gov_investment',
 'subsidy_date',
 'target_inno_subsidy',
 'legal_form_binary_3.0',
 'legal_form_binary_4.0',
 'legal_form_binary_5.0',
 'legal_form_binary_6.0',
 'legal_form_binary_8.0',
 'legal_form_binary_10.0',
 'legal_form_binary_12.0',
 'legal_form_binary_13.0',
 'legal_form_binary_15.0',
 'legal_form_binary_16.

In [ ]:
class ModelEvaluation:

    def __init__(self, target_vars: list[str], model_specs: dict, random_state: int = 42):
        """
        target_vars: list of target column names
        model_specs: dict with keys as model names, values as dicts with:
                     - 'model': a scikit-learn estimator class or a callable returning an instance
                     - 'param_grid': dict of hyperparameter grid for grid search
        Example:
        model_specs = {
            'logreg': {'model': LogisticRegression, 'param_grid': {...}},
            'rf': {'model': RandomForestClassifier, 'param_grid': {...}}
        }
        """
        self.target_vars = target_vars
        self.model_specs = model_specs
        self.random_state = random_state
        self.best_params = {m: {t: None for t in target_vars} for m in model_specs}
        self.selected_features = {m: {t: None for t in target_vars} for m in model_specs}
        self.best_models = {m: {t: None for t in target_vars} for m in model_specs}
        self.production_models = {m: {t: None for t in target_vars} for m in model_specs}
        self.metrics_report = {m: {t: None for t in target_vars} for m in model_specs}

    def load_data(self, df: pd.DataFrame, feature_cols: list[str]):
        self.df = df.copy()
        self.feature_cols = feature_cols

    def load_best_params(self, file_path: Path, model_names: list[str], target_vars: list[str]):
        with open(file_path, 'rb') as f:
            all_params = pickle.load(f)
        self.best_params = {
            m: {t: all_params[m][t] for t in target_vars}
            for m in model_names
        }

    def load_best_features(self, file_path: Path, model_names: list[str], target_vars: list[str], additional_features: list = []):
        with open(file_path, 'rb') as f:
            all_features = pickle.load(f)
        self.selected_features = {
            m: {t: all_features[m][t] + additional_features for t in target_vars}
            for m in model_names
        }

    def _get_feature_importances(self, model):
        """Return feature importances as a pandas Series, sorted descending."""
        # Tree-based models
        if hasattr(model, "feature_importances_"):
            return model.feature_importances_
        # Linear models (coefficients)
        elif hasattr(model, "coef_"):
            return np.abs(model.coef_).flatten()
        else:
            return None

    def _add_class_weights_to_fit_params(self, fit_params, ModelClass, model_name, y):
        
        if not self.model_specs[model_name]['include_class_weights']:
            return fit_params

        classes = np.unique(y)

        if model_name == 'xgb':
            n_pos = np.sum(y == classes[1])
            n_neg = np.sum(y == classes[0])
            fit_params['scale_pos_weight'] = n_neg / n_pos if n_pos > 0 else 1.0

        elif hasattr(ModelClass(), 'class_weight'):
            class_weights = compute_class_weight('balanced', classes=classes, y=y)
            cw_dict = {cls: w for cls, w in zip(classes, class_weights)}
            fit_params['class_weight'] = cw_dict

        return fit_params

    def nested_cv_with_feature_selection(
        self,
        k_outer: int = 5,
        k_inner: int = 5,
        min_features_to_select: int = 3,
        scoring: str = "roc_auc"
    ):
        """
        Perform feature selection (RFECV) using cross-validation without hyperparameter tuning.

        Saves the most frequently selected features across outer folds.
        """

        for model_name, spec in self.model_specs.items():
            ModelClass = spec['model']
            fit_params = self._add_class_weights_to_fit_params(spec['fit_params'])

            for target in self.target_vars:
                X = self.df[self.feature_cols].values
                y = self.df[target].values

                outer_skf = StratifiedKFold(n_splits=k_outer, shuffle=True, random_state=self.random_state)

                selected_feature_masks = []

                for train_idx, _ in outer_skf.split(X, y):
                    X_train, y_train = X[train_idx], y[train_idx]

                    model = ModelClass(**fit_params)
                    
                    inner_skf = StratifiedKFold(n_splits=k_inner, shuffle=True, random_state=self.random_state)
                    rfecv = RFECV(
                        estimator=model,
                        step=1,
                        min_features_to_select=min_features_to_select,
                        cv=inner_skf,
                        scoring=scoring,
                        n_jobs=-1
                    )

                    rfecv.fit(X_train, y_train)
                    selected_feature_masks.append(rfecv.support_)

                # Aggregate selected features across folds
                selected_feature_masks = np.array(selected_feature_masks)
                mean_mask = selected_feature_masks.mean(axis=0)
                threshold = 0.6  # at least 60% of folds must have selected a feature
                final_mask = mean_mask >= threshold
                final_features = np.array(self.feature_cols)[final_mask].tolist()

                self.selected_features[model_name][target] = final_features
                print(f"[{model_name}/{target}] Selected {len(final_features)} features: {final_features}")

    def nested_cv_with_hyperparam_search(self, out_folder: Path, k_outer: int = 5, k_inner: int = 5, best_features: bool = False):
        
        if best_features and not self.selected_features:
            raise ValueError('To use the best features execure find_best_feature_subset first!')
        
        for model_name, spec in self.model_specs.items():

            ModelClass = spec['model']
            fit_params = self._add_class_weights_to_fit_params(spec['fit_params'])
            param_grid = spec['param_grid']

            for target in self.target_vars:
                features = self.selected_features[model_name][target] if best_features else self.feature_cols
                X = self.df[features].values
                y = self.df[target].values

                outer_skf = StratifiedKFold(n_splits=k_outer, shuffle=True, random_state=self.random_state)
                
                outer_aucs = []
                feature_importances_folds = []
                inner_best_params = []
                
                # Outer loop over k_outer folds
                for train_idx, test_idx in outer_skf.split(X, y):
                    
                    # Init data of the current outer fold
                    X_train, X_test = X[train_idx], X[test_idx]
                    y_train, y_test = y[train_idx], y[test_idx]
                    
                    # Only include relevant fit_params for this model
                    model = ModelClass(**fit_params)

                    # Set up inner grid search loop for k_inner folds
                    inner_skf = StratifiedKFold(n_splits=k_inner, shuffle=True, random_state=self.random_state)
                    grid_search = GridSearchCV(
                        estimator=model,
                        param_grid=param_grid,
                        cv=inner_skf,
                        scoring='roc_auc',
                        n_jobs=-1
                    )
                    
                    # Determine best hyperparameters of this fold using only the training data and not testing
                    # training data is then again split into k_inner folds
                    grid_search.fit(X_train, y_train)
                    
                    # Select and store best hyperparam config determined on the training data
                    inner_lambda_star = grid_search.best_params_
                    inner_best_params.append(inner_lambda_star)
                    
                    # Refit model with full training data to estimate auc and feature importance of the outer fold
                    temp_fit_params = {**fit_params, 'n_jobs': -1}  # For training without CV set to -1
                    params = dict(inner_lambda_star, **temp_fit_params)
                    best_model = ModelClass(**params)
                    best_model.fit(X_train, y_train)
                    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
                    
                    feature_importances_folds.append(self._get_feature_importances(best_model))
                    auc = roc_auc_score(y_test, y_pred_proba)
                    outer_aucs.append(auc)

                mean_feature_importance = None
                if feature_importances_folds:
                    mean_importances = np.mean(feature_importances_folds, axis=0)
                    mean_feature_importance = {feature: score for feature, score in zip(features, mean_importances)}

                # Select hyperparameters from the inner folds that were most frequently picked as the best hyperparameters for the outer model
                overall_lambda_star = Counter([frozenset(p.items()) for p in inner_best_params]).most_common(1)[0][0]
                overall_lambda_star = dict(overall_lambda_star)

                # Retrain final model on all data
                temp_fit_params = {**fit_params, 'n_jobs': -1}  # For training without CV set to -1
                best_params = dict(overall_lambda_star, **temp_fit_params)
                production_model = ModelClass(**best_params)
                production_model.fit(X, y)

                # Store production model
                self.best_models[model_name][target] = production_model

                # Store best hyperparameters
                self.best_params[model_name][target] = overall_lambda_star

                # Store metrics report for the avg performance of the model on the current target
                self.metrics_report[model_name][target] = {
                    'mean_auc': np.mean(outer_aucs),
                    'std_auc': np.std(outer_aucs),
                    'all_auc': outer_aucs,
                    'best_params': overall_lambda_star,
                    'mean_feature_importances': mean_feature_importance
                }
        self._save_models_and_reports(out_folder)

    def _save_models_and_reports(self, out_folder: Path):
        """Retrains model on the full dataset and stores production ready model with additional performance reports
        from the k-fold CV evaluations."""
        
        out_folder.mkdir(parents=True, exist_ok=True)

        # Save best models stored in self.best_models[model_name][target]
        models_dir = out_folder / 'trained_models'
        models_dir.mkdir(exist_ok=True)
        for model_name, targets in self.best_models.items():
            for target, model in targets.items():
                model_path = models_dir / f'{model_name}_{target}.joblib'
                joblib.dump(model, model_path)
                print(f"Saved model for {model_name}/{target} to {model_path}")

        # Save metrics report as csv file
        summary_rows = []
        for model_name, model_results in self.metrics_report.items():
            for target, d in model_results.items():
                summary_rows.append({
                    'model': model_name,
                    'target': target,
                    'mean_auc': d['mean_auc'],
                    'std_auc': d['std_auc'],
                    'all_auc': str(d['all_auc']),
                    'best_params': str(d['best_params']),
                    'mean_feature_importances': str(d['mean_feature_importances'])
                })
        pd.DataFrame(summary_rows).to_csv(out_folder / 'cv_metrics_report.csv', index=False)
        print(f"Saved metrics summary to {str(out_folder / 'cv_metrics_report.csv')}")

        # Save hyperparameters
        best_params_path = out_folder / 'best_hyperparameters.pkl'
        with open(best_params_path, 'wb') as f:
            pickle.dump(self.best_params, f)
        print(f"Saved best hyperparameters to {best_params_path}")

        # Save best feature set
        best_features_path = out_folder / 'best_features.pkl'
        with open(best_features_path, 'wb') as f:
            pickle.dump(self.selected_features, f)
        print(f"Saved best features to {best_features_path}")

In [ ]:
RANDOM_STATE = 42


MODEL_SPECS = {
    'logreg': {
        'model': LogisticRegression,
        'fit_params': {
            'random_state': RANDOM_STATE,
            'n_jobs': 1,  # Avoid thread thrashing, so model n_jobs should be set to 1 because Grid Search CV and Feature Selection is set to -1
            'max_iter': 1000,
            'solver': 'saga',  # Fixed for computational efficiency
        },
        'param_grid': [
            {
                'penalty': ['none'],  # Test vanilla LogReg
            },
            {
                'penalty': ['l1', 'l2'],  # Test Lasso and Ridge regularization
                'C': [0.001, 0.01, 0.1, 1, 10, 100],
            },
        ],
        'include_class_weights': True
    },
    'rf': {
        'model': RandomForestClassifier,
        'fit_params': {
            'random_state': RANDOM_STATE,
            'n_jobs': 1,
        },
        'param_grid': {
            'n_estimators': [100, 300, 500],
            'max_depth': [None, 10, 25, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', 0.5],
            'criterion': ['gini', 'entropy'],
        },
        'include_class_weights': True
    },
    'xgb': {
        'model': XGBClassifier,
        'fit_params': {
            'random_state': RANDOM_STATE,
            'n_jobs': 1,
            'objective': 'binary:logistic',
            'verbosity': 0,
            'booster': 'gbtree',
            'tree_method': 'hist',
            'use_label_encoder': False,
            'eval_metric': 'auc',
        },
        'param_grid': {
            'n_estimators': [100, 300],
            'max_depth': [3, 6],
            'learning_rate': [0.01, 0.1],
            'subsample': [0.8, 1.0],
            'colsample_bytree': [0.8, 1.0],
            'min_child_weight': [1, 5],
            'gamma': [0, 1],
            'reg_alpha': [0, 1],
            'reg_lambda': [1, 2],
        },
        'include_class_weights': True
    }
}

In [ ]:
"""
Experiment A: Reproducability experiment and model evaluation on the full data set without website data
"""

# 1. Load data for experiment A
df_a = pd.read_csv(PROCESSED_DATA_DIR / ...)

FEATURE_COLS = [col for col in df_a.columns if col not in ['target1', 'target2', 'target3', 'target4']]
TARGET_VARS = ['target1', 'target2', 'target3', 'target4']

# 2. Initialize model evaluation with targets and model specs
meval = ModelEvaluation(TARGET_VARS, MODEL_SPECS, random_state=RANDOM_STATE)
meval.load_data(df_a, FEATURE_COLS)

In [ ]:
# 1. Training procedure on all features for the baseline reproduction
out_folder = MODELS_DIR / 'experiment_A'
meval.nested_cv_with_hyperparam_search(out_folder=out_folder)

In [ ]:
"""
Experiment B: Performance difference between Doc2vec and my implementation
"""

BEST_MODEL = 'xgb'
TARGET_VARS = [...]


MODELS = {
    'logreg': LogisticRegression,
    'rf': RandomForestClassifier,
    'xgb': XGBClassifier,
}

# 1. Load data for experiment B
df_b = pd.read_csv(PROCESSED_DATA_DIR / ...)
FEATURE_COLS = [col for col in df_b.columns if col not in ['target1', 'target2', 'target3', 'target4']]

# 2. Initialize model with best best performing model from experiment A
model_specs = {
    BEST_MODEL: {
        'model': MODELS.get(BEST_MODEL),
        'fit_params': {
            'random_state': RANDOM_STATE,
            'n_jobs': -1
            'max_iter': 1000
        },
        'param_grid': [
            {
                'penalty': ['none'],
                'solver': ['saga'],
            },
            {
                'penalty': ['l1'],
                'solver': ['saga'],
                'C': [0.01, 0.1, 1, 10],
            },
            {
                'penalty': ['l2'],
                'solver': ['saga'],
                'C': [0.01, 0.1, 1, 10],
            },
        ],
        'include_class_weights': True
    }
}
meval = ModelEvaluation(TARGET_VARS, model_specs)
meval.load_data(df_b, FEATURE_COLS)

# 3. Use best model params and features from experiment A
exp_A_folder = MODELS_DIR / 'experiment_A'

meval.load_best_params(exp_A_folder / 'best_hyperparameters.pkl', ['xgb'], TARGET_VARS)

In [ ]:
# 1. Evaluate with doc2vec scores
out_folder = MODELS_DIR / 'experiment_B' / 'doc2vec'

meval.nested_cv_with_hyperparam_search(out_folder=out_folder)

# 2. Evaluate with dimension scores
out_folder = MODELS_DIR / 'experiment_B' / 'dimensions'

meval.nested_cv_with_hyperparam_search(out_folder=out_folder)

In [ ]:
"""
Experiment C: Performance difference between Doc2vec and my implementation
"""

# Add hyperparams to fit_params for the initial feature selection without tuning
model_specs = {
    BEST_MODEL: {
        'model': MODELS.get(BEST_MODEL),
        'fit_params': {
            'random_state': RANDOM_STATE,
            'n_jobs': -1
            'max_iter': 1000
        },
        'param_grid': [
            {
                'penalty': ['none'],
                'solver': ['saga'],
            },
            {
                'penalty': ['l1'],
                'solver': ['saga'],
                'C': [0.01, 0.1, 1, 10],
            },
            {
                'penalty': ['l2'],
                'solver': ['saga'],
                'C': [0.01, 0.1, 1, 10],
            },
        ],
        'include_class_weights': True
    }
}


# 1. Init model evaluation with same setup as in experiment B
meval = ModelEvaluation(TARGET_VARS, model_specs)
meval.load_data(df_b, FEATURE_COLS)

# 2. Training procedure with feature selection
out_folder = MODELS_DIR / 'experiment_C'

meval.nested_cv_with_feature_selection()
meval.nested_cv_with_hyperparam_search(out_folder=out_folder, best_features=True)